### Defining Random Variables in =python=                       :code_example:



The distinguishing feature of variables in a field such as the
reals or the complex plane is their *value*; the distinguishing
feature of random variables is their *distribution*.  The `python`
package `scipy.stats` is well-engineered and offers many different
distributions, and tools to construct others, while the package
`pacal` is perhaps less well engineered, but defines arithmetic
operations over random variables which allows for more elegant
semantics.

There are two main classes of random variables to consider: discrete
and continuous.  The distinction is worth drawing because different
classes are handled differently in many mathematical operations.  

For example, here we instantiate a scalar $\rvx$:



In [1]:
from pacal import NormalDistr

x = NormalDistr()

And here we instantiate a discrete random variable which is defined
 over an event space $\{-1,0,1\}$ with corresponding probabilities $(1/3,1/2,1/6)$:



In [1]:
from pacal import DiscreteDistr

Omega = (-1,0,1)
Pr = (1/3.,1/2.,1/6.)

s = DiscreteDistr(Omega,Pr)

:results:
 :end:

Now, here are some things we can do with these random variables.
 First, the continuous  $\rvx$:



In [1]:
print("E(x) = %6.4f" % x.mean())
print()
print("Some (central) moments of x:")
print([(m,x.moment(m)) for m in [1,2,3,4]])
print()
print("95%% confidence interval: (%f,%f)" % x.interval(0.95))
print()
print(x.cdf(0),x.pdf(0))

Next, the discrete r.v., \rv{s}:



In [1]:
print("E(s) = %6.4f" % s.mean())
print()
print("Some moments of x:")
print([(m,s.moment(m)) for m in [1,2,3,4]])
print()
print("95%% confidence interval: (%f,%f)" % s.interval(0.95))
print()
# Note! More accurately not pdf, but pmf for discrete rv.
print(s.cdf(0),s.pdf(0))

If we want *realizations* of these random variables:



In [1]:
N=3
print(x.rand(N))

And we can also combine different random variables&#x2014;which yields a
new random variable.  For instance,



In [1]:
y = x + s

What do you suppose the pdf of \rvy looks like?



In [1]:
import plotly.graph_objects as go
import numpy as np

X = np.linspace(-2,3,100).tolist()

fig = go.Figure(data=go.Scatter(x=X, y=[y.pdf(z) for z in X]))
fig.show()

#### Exercise



Prove that \rvy is continuous (in the sense that it has a density),
     as suggested by the figure *or* establish that the figure is
     wrong or misleading.

